# Fragments table: field purposes

* **id (INTEGER PRIMARY KEY AUTOINCREMENT)**
  Internal row key. Unique per row. Never used for dedup logic—just a stable handle for joins/FTS rowid.

* **server_id (INTEGER NOT NULL)**
  Discord guild provenance. Lets you scope searches by server and do retention per server.

* **channel_id (INTEGER NOT NULL)**
  Discord channel provenance. Scopes searches/retention and helps rebuild context threads.

* **message_id (INTEGER NOT NULL)**
  Discord message provenance. All fragments from the same message share this. Central to idempotent ingest (see unique key with `source_idx`).

* **author_id (INTEGER NOT NULL)**
  Provenance/analytics. Useful for per-user summaries, style, or filtering.

* **ts (REAL NOT NULL)**
  Unix timestamp (seconds). Enables recency ranking and time-based pruning.

* **content (TEXT NOT NULL)**
  The semantic text you embed/search (clean—no modality prefixes). This is what FTS and embeddings run on.

* **type (TEXT NOT NULL)**
  Modality of the fragment (e.g., `"text" | "image" | "youtube" | "link" | "gif"`).
  Used for filtering/boosting and included in `content_hash` to avoid cross-modality collisions.

* **title (TEXT)**
  Optional UX metadata (nice heading/snippet). Not part of identity.

* **url (TEXT)**
  Optional deep-link back to source (e.g., YouTube URL, external link, Discord CDN URL). Not part of identity; useful for previews and rehydration.

* **media_id (TEXT NOT NULL)**
  **Deterministic asset identity** across messages/channels/servers.
  Examples: `yt:<video_id>`, `dc:<attachments/...>`, `url:<blake16(normalized_url)>`, or `msg:<message_id>:<source_idx>:<type>` for text.
  Use for asset-level dedupe, analytics, policy (block/allow), and joining richer metadata.
  *Note:* Not included in the `UNIQUE(...)` constraint so you can still store multiple captions for the same asset if desired.

* **embedding (BLOB NOT NULL)**
  Float32 vector bytes. Used for vector re-ranking after optional FTS prefilter.

* **emb_model (TEXT NOT NULL)**
  Embedding model identifier (e.g., `"text-embedding-3-small"`). Stored so embeddings can be migrated or compared across models.

* **emb_dim (INTEGER NOT NULL)**
  Dimensionality of the stored embedding (e.g., `1536`). Helps validate vector sizes.

* **source_idx (INTEGER NOT NULL)**
  The fragment’s index within the message (`fragments[i]`). Disambiguates multiple fragments under the same `message_id`. Critical for idempotent ingest.

* **content_hash (TEXT NOT NULL)**
  Hash of the semantic payload (e.g., `blake16(f"{type}:{content}")`).
  Used in the uniqueness constraint to avoid re-inserting the exact same fragment for the same message/index/modality.

# Uniqueness & dedup strategy

* **UNIQUE(message_id, source_idx, type, content_hash)**
  Guarantees *idempotent ingest per message*: if you re-ingest the same message, the same fragments (same position, same modality, same text) are upserted, not duplicated.

  * Protects against duplicates caused by retries/rebuilds.
  * Allows different fragments in the same message (different `source_idx`) and different modalities (`type`).
  * Allows different captions for the same media (different `content_hash`)—by design.

* **media_id (NOT NULL, optionally indexed)**
  Not part of the UNIQUE key, so you can store **multiple rows** that reference the same asset (e.g., same YouTube video with different captions across time/channels).
  Fast cross-message asset queries/dedupe with:
  `CREATE INDEX IF NOT EXISTS idx_frag_media ON fragments(media_id);`

# Typical query/index helpers (optional but recommended)

* `CREATE INDEX IF NOT EXISTS idx_frag_sc  ON fragments(server_id, channel_id);`
  Fast server/channel scoping.

* `CREATE INDEX IF NOT EXISTS idx_frag_ts  ON fragments(ts);`
  Fast recency windows and pruning.

* `CREATE INDEX IF NOT EXISTS idx_frag_msg ON fragments(message_id);`
  Fast fetch-by-message (e.g., rebuilding message context).

* `CREATE INDEX IF NOT EXISTS idx_frag_media ON fragments(media_id);`
  Fast asset-level operations (analytics, dedupe, policy).

# Why both `content_hash` and `media_id`?

* **content_hash** → identity of the **text payload** (caption/description/message).

  * Prevents duplicate fragments during re-ingest of the *same message*.
  * Changes if caption/description changes.

* **media_id** → identity of the **underlying asset** (video/image/link).

  * Stable across messages, channels, servers.
  * Same even if captions change.
  * Enables cross-message dedupe/aggregation/joining.

# Minimal mental model

* **Provenance:** `server_id, channel_id, message_id, author_id, ts`
* **Semantics:** `type, content, embedding` + `emb_model`, `emb_dim`
* **Idempotence (within a message):** `message_id + source_idx + type + content_hash`
* **Asset identity (cross-message):** `media_id`
* **UX metadata:** `title, url`

# Supporting tables

* **fragments_fts** — FTS5 table mirroring `fragments.content` for lexical prefilter.
* **user_profiles** — Per-user JSON blobs.
* **server_styles** — Server-level style guide blobs.
* **channel_summaries** — Per-channel summary text.
